# Oracles Tutorial

Quantum oracles are a key part of many quantum algorithms and often we assume that we are provided a quantum oracle to implement some function.  This tutorial has been created to increase your familiarity with quantum oracles, their properties, and the different types as well as some applications of the oracles.

This tutorial will:
* introduce you to quantum oracles and how they relate to classical oracles
* provide mathematical examples of how quantum oracles act on quantum states
    * introduction to **phase kickback** and why it is an important consequence of quantum oracles
* teach you how to implement quantum oracles in Q#
* show you how you could test a quantum oracle that you've implemented in industry

Before we jump into the material, we suggest that you are familiar with the following topics:
* quantum [qubits](https://github.com/microsoft/QuantumKatas/tree/main/tutorials/Qubit) and how they're used in Q#
* [linear algebra](https://github.com/microsoft/QuantumKatas/blob/main/tutorials/LinearAlgebra) properties of quantum operators
* [basic quantum computing gates](https://github.com/microsoft/QuantumKatas/blob/main/BasicGates) as quantum oracles are generally composed with these gates
* [superposition](https://github.com/microsoft/QuantumKatas/blob/main/Superposition)

Let's get started!

# Part I - Introduction to Quantum Oracles

## Classical Oracles
In classical computing, we often discuss black box versus white box testing.  In white box testing, the implementation of a function, say to factor a number, is visible to the tester.  Thus the tester can test specific expectations such as runtime or memory complexity.  However, in black box testing, the tester can only test the functionality and expected behavior of the function - the implementation has been abstracted away.

Generally, most functions and data structures that we use while programming are a black box in our eyes - we are not worried with how an linked list is implemented, we only care about the functionality that it provides.  

Suppose I provided you the following function:
The function takes two parameters as input, two lists, where these lists represent the availability during the week of two different employees at a company.  The function returns if true if there is a day (Monday, Tuesday, Wednesday, Thursday, or Friday) for which they are both free and could schedule a meeting, if no such date exists then the function returns false.

The previous proposition is an example of a **classical oracle** because when interacting with this function, you are not worried with *how* the function determines the answer, you only care about *what* the answer is when provided a specific input.

Formally, a **classical oracle** is a function that provided some input, it produces a *deterministic* output.  Hence the same input *always* results in the same output.

### Task 1.1: Implement a classical oracle
**Inputs:**

  1. An array of booleans capturing the bit representation of a number

**Goal:**

Determine if the input array is representing the bit pattern of
the number seven, returning true if it is, and false otherwise.

**Examples:**

* If the input array is [false, true, true, true], then return true.

* If the input array is [true, true, true, false], then return false.

* if the input array is [true, false, true, fakse, true, false], then return false.

In [ ]:
%kata T11_IsSeven_ClassicalOracle

function IsSeven(x: Bool[]) : Bool {
    // ...
    return false;
}

## Quantum Oracles

An oracle in the quantum world is a black box operation that is utilized during an algorithm.  These **quantum oracles** are passed as input to quantum algorithms and they themselves require inputs to operate on.  A quantum oracle implements some function $f: \{0,1\}^n \rightarrow \{0,1\}^m$ where $x$ is the input state of the form $|x\rangle = |x_0\rangle \otimes |x_1\rangle \otimes ... \otimes |x_{n-1}\rangle$.  Many quantum algorithms assume a proper implementation of some quantum oracle as input, but this is a very strong assumption.  Throughout the rest of this tutorial you will be exposed to quantum oracles and learn how to implement them.

There are two types of quantum oracles: phase oracles and marking oracles.  Each of these types oracles are defined based on their operations on their basis states, where these basis states are always $|0\rangle$ and $|1\rangle$.  Oracles must be unitary as well as not change the input if that input is a basis state of the oracle.  Oracles also follow the same rules of linear algebra as normal matrices as they themselves are matrices, refer to the intro to review some of the properties of quantum operators (matrices).

### Phase Oracles
A phase oracle $U_{phase}$ is an oracle that when provided some state $|x\rangle$ it flips the sign of that state if $f(x)=1$.  Thus:

$$U_{phase} |x\rangle = (-1)^{f(|x\rangle)}|x\rangle$$

### Demo 1.1: Phase quantum oracle - alternating bit pattern!
Consider a phase oracle defined as follows:

**Inputs:**

  1. 3 qubits in an arbitrary state $|x\rangle$ (input/query register)

**Goal:**

Flip the sign of the input state $|x\rangle$ if the input register is in
the state $|101\rangle$ or $|010\rangle$ and leave the input register
unchanged otherwise.

**Examples:**

* If the query register is in the state $|010\rangle$, flip its sign.

* If the query register is in the state $|101\rangle$, flip its sign.

* If the query register is in the state $|111\rangle$, do nothing.

* If the query register is in the state $|001\rangle$, do nothing.

In [ ]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Diagnostics;

operation AlternatingBitPattern_PhaseOracle(x: Qubit[]) : Unit is Adj + Ctl {
    let PatternOne = ControlledOnBitString([false, true, false], Z);
    let PatternTwo = ControlledOnBitString([true, false, true], Z);
    using(q = Qubit()) {
        X(q);
        PatternOne(x, q);
        PatternTwo(x, q);
        X(q);
    }
}

operation PhaseOracle_Demo() : Unit {
    let divider = "--------------------------------------------------------------------------------------------------";
    
    // allocate the |000⟩ state
    using(q = Qubit[3]) {
        // Prepare an equal superposition of all basis states
        ApplyToEachA(H, q);
        
        // Print out that we currently have a superposition of all the basis states
        // Notice the phases on each of the components
        Message("State |β⟩ (equal superposition of all basis states):");
        DumpMachine();
        Message(divider);
        
        // Apply the oracle
        AlternatingBitPattern_PhaseOracle(q);
        
        
        // Print out the resulting state of AND |β⟩
        // Notice that the phase of the state is still positive
        Message("State after applying the alternating phase oracle to |β⟩:");
        DumpMachine();
        Message(divider);
        
        // Reset our state q back to all zeros for deallocation
        ResetAll(q);
    }
}

In [ ]:
%simulate PhaseOracle_Demo

> Notice that the input state above is an equal superposition and after applying the oracle we have the same equal superposition except that the states $|010\rangle$ and $|101\rangle$ had their phase flipped!  Recall that these two states are exactly the two states for which $f(x) = 1$ for the alternating bit pattern phase oracle $U_{alt,phase}$, thus they are exactly the two states we expect to experience a phase flip!

Now you will implement the same classical oracle that you implemented in <span style="color:blue">Exercise 1</span> as a quantum phase oracle.

### Task 1.2: Implement a phase quantum oracle
Consider a phase oracle defined as follows (call this oracle $U_{7,phase}$):

**Inputs:**

  1. 3 qubits in an arbitrary state $|x\rangle$ (input/query register)

**Goal:**

Flip the sign of the input state $|x\rangle$ if the input register is in
the state $|111\rangle$ and leave the input register unchanged otherwise.

**Examples:**

* If the query register is in the state $|010\rangle$, do nothing.

* If the query register is in the state $|101\rangle$, do nothing.

* If the query register is in the state $|111\rangle$, flip its sign.

* If the query register is in the state $|001\rangle$, do nothing.

> **Don't allocate auxiliary qubits to perform this operation.**

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  To solve this problem, you need to find a gate that will only flip the sign of the $|111\rangle$ basis state.  Which single-qubit gate flips the sign of the basis state $|1\rangle$ but not $|0\rangle$, how can you modify this gate to solve this problem?
</details>

In [ ]:
%kata T12_IsSeven_PhaseOracle 

operation IsSeven_PhaseOracle (x : Qubit[]) : Unit is Adj + Ctl {
    // ...
}

#### Mathematical properties

Consider how the oracle from <span style="color:blue">Exercise 2</span> acts on its basis states:
$$U_{7,phase} |111\rangle = -|111\rangle$$
$$U_{7,phase} |110\rangle = |110\rangle$$

This follows the requirement that a $U_{7,phase}$ does not change the input if it's a basis state as well as the fact that $U_{7,phase}$ does not change the norm of the state ($U_{7,phase}$ is unitary).  However, consider if we were provided a state in superposition instead, what might that look like?

Suppose that $|\beta\rangle$ is an equal superposition of the $6$ and $7$ state: 
$$|\beta\rangle = \frac{1}{\sqrt{2}} \big(|110\rangle + |111\rangle\big) = |11\rangle \otimes \frac{1}{\sqrt{2}} \big(|0\rangle + |1\rangle\big) = |11\rangle \otimes |+\rangle = |11+\rangle$$

Let's consider how our operator acts on this new state:
$$U_{7,phase} |\beta\rangle = U_{7,phase} \Big[\frac{1}{\sqrt{2}} \big(|110\rangle + |111\rangle\big)\Big] = $$
$$= \frac{1}{\sqrt{2}} \big(U_{7,phase} |110\rangle + U_{7,phase} |111\rangle\big) = \frac{1}{\sqrt{2}} \big(|110\rangle - |111\rangle\big) := |\gamma\rangle$$

Now the question is if our input state was modified during this operation; lets simplify $|\gamma\rangle$:
$$|\gamma\rangle = \frac{1}{\sqrt{2}} \big(|110\rangle - |111\rangle\big) = |11\rangle \otimes \frac{1}{\sqrt{2}} \big(|0\rangle - |1\rangle\big) = $$
$$= |11\rangle \otimes |-\rangle = |11-\rangle \neq |\beta\rangle$$

Here we see that the oracle modifies the input.  This is conditioned on the input state being a *superposition* of the basis states of the oracle - as a phase oracle will only modify the sign of its basis states.

> It is also worth noting that while the oracle modified the input when provided a superposition state, it did *not* modify the norm of that state.  As an exercise, you can verify this yourself by taking the norm of $|\beta\rangle$ and $|\gamma\rangle$, which both will result in a value of $1$.
>
>
> As another exercise, consider how you could distinguish between the input and output state programmatically?  Is there an operation that you could apply to the initial state $|\beta\rangle$ and the final state $|\gamma\rangle$ to show that the two states are not equivalent?

### Marking Oracles

A marking oracle $U_{mark}$ is an oracle that when provided some state $|x\rangle$ and some qubit $y$ performs addition modulo 2 between $f(x)$ and $y$.  Hence $U_{mark}$ is an operator that performs the following operation:

$$U_{mark}|x\rangle |y\rangle = U_{mark}\big(|x\rangle \otimes |y\rangle\big) = |x\rangle \otimes |y \oplus f(x)\rangle = |x\rangle |y \oplus f(x)\rangle$$

### Demo 1.2: Marking quantum oracle - alternating bit pattern!
Consider a marking oracle defined as follows:

**Inputs:**

  1. 3 qubits in an arbitrary state $|x\rangle$ (input/query register)
    
  2. A qubit in an arbitrary state $|y\rangle$ (target qubit)

**Goal:**

Flip the state of $|y\rangle$ if the input register is in the 
state $|101\rangle$ or $|010\rangle$ and leave the state $|y\rangle$
unchanged otherwise.

**Examples:**

* If the query register is in the state $|010\rangle$, flip the state $|y\rangle$.

* If the query register is in the state $|101\rangle$, flip the state $|y\rangle$.

* If the query register is in the state $|111\rangle$, do nothing.

* If the query register is in the state $|001\rangle$, do nothing.

In [ ]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Diagnostics;

operation AlternatingBitPattern_MarkingOracle(x: Qubit[], y: Qubit) : Unit is Adj + Ctl {
    let PatternOne = ControlledOnBitString([false, true, false], X);
    let PatternTwo = ControlledOnBitString([true, false, true], X);
    PatternOne(x, y);
    PatternTwo(x, y);
}

operation MarkingOracle_Demo() : Unit {
    let divider = "--------------------------------------------------------------------------------------------------";
    
    // allocate the |000⟩|0⟩ state
    using(q = Qubit[4]) {
        // Prepare an unequal superposition of all basis states
        //PrepareArbitraryStateD(H, q[0..Length(q)-2]);
        ApplyToEachA(H, q[0..Length(q)-2]);
        
        // Print out that we currently have an unequal superposition of all the 
        // basis states.  Notice the phases on each of the components
        Message("State |β⟩ (equal superposition of all basis states):");
        DumpMachine();
        Message(divider);
        
        // Apply the oracle
        AlternatingBitPattern_MarkingOracle(q[0..Length(q)-2], q[Length(q)-1]);
        
        // Print out the resulting state of AND |β⟩
        // Notice that the phase of the state is still positive
        Message("State after applying the alterating marking oracle to |β⟩:");
        DumpMachine();
        Message(divider);
        
        // Reset our state q back to all zeros for deallocation
        ResetAll(q);
    }
}

In [ ]:
%simulate MarkingOracle_Demo

> Let's compare the initial state to the final state from the above demo.  In the initial state we had an equal superposition tensored with the state $|0\rangle$.  In the final state, this is no longer the case.  Two of the basis states from the initial equal superposition are now tensored with $|1\rangle$ instead of $|0\rangle$.  These two basis states from the initial state are $|010\rangle$ and $|101\rangle$.  Now in the final state we see that $|0100\rangle$ and $|1010\rangle$ no longer have a weight associated with them, instead $|0101\rangle$ and $|1011\rangle$ have non-zero weights in the final state - both of which had zero weight in the initial state.
>
> This is exactly the result that we expect.  Recall our function $f(x)$: $f(x)=1$ if and only if $|x\rangle=|010\rangle$ or $|x\rangle=|101\rangle$.  Also recall that the first three qubits of the allocated qubits represent $|x\rangle$ and the last represents $|y\rangle$.  Thus when we have the two basis states, $|x\rangle=|010\rangle$ or $|x\rangle=|101\rangle$, we will flip the qubit $|y\rangle$, causing these two initial states to be tensored with $|1\rangle$ in the final state where originally they were tensored with $|0\rangle$.
>
> In the initial state, both $|0100\rangle$ and $|1010\rangle$ cause $f(x)$ to evaluate to $1$ thus flipping the fourth qubit $|0\rangle$ to $|1\rangle$.  Applying the oracle, $U_{alt,mark}|0100\rangle = |0101\rangle$ and $U_{alt,mark}|1010\rangle = |1011\rangle$ where all other basis states in the initial equal superposition remain unchanged.

Now you will implement the same quantum oracle that you implemented in <span style="color:blue">Exercise 2</span> but this time as a marking oracle instead.

### Task 1.3: Implement a marking quantum oracle
Consider a marking oracle defined as follows (call this oracle $U_{7,mark}$):

**Inputs:**

  1. 3 qubits in an arbitrary state $|x\rangle$ (input/query register)
    
  2. A qubit in an arbitrary state $|y\rangle$ (target qubit)

**Goal:**

Flip the state of $|y\rangle$ if the input register is in the 
state $|111\rangle$ and leave the state $|y\rangle$ unchanged otherwise.

**Examples:**

* If the query register is in the state $|010\rangle$, do nothing.

* If the query register is in the state $|101\rangle$, do nothing.

* If the query register is in the state $|111\rangle$, flip the state $|y\rangle$.

* If the query register is in the state $|001\rangle$, do nothing.

In [ ]:
%kata T13_IsSeven_MarkingOracle 

operation IsSeven_MarkingOracle(x: Qubit[], y: Qubit) : Unit is Adj + Ctl {
    // ...
}

#### Mathematical Properties

Again, let us consider how the marking oracle that you just implemented in <span style="color:blue">Exercise 3</span> affects the input when the input is the oracles basis states opposed to a superposition state.
$$U_{7,mark} |111\rangle |0\rangle = |111\rangle |0 \oplus f(111)\rangle = |111\rangle |0 \oplus 1\rangle = |111\rangle |1\rangle$$
$$U_{7,mark} |111\rangle |1\rangle = |111\rangle |1 \oplus f(111)\rangle = |111\rangle |1 \oplus 1\rangle = |111\rangle |0\rangle$$

$$U_{7,mark} |110\rangle |0\rangle = |110\rangle |0 \oplus f(110)\rangle = |110\rangle |0 \oplus 0\rangle = |110\rangle |0\rangle$$
$$U_{7,mark} |110\rangle |1\rangle = |110\rangle |1 \oplus f(110)\rangle = |110\rangle |1 \oplus 0\rangle = |110\rangle |1\rangle$$

In the previous examples, all of the input $|x\rangle$ and $|y\rangle$ are in basis states of the oracle $U_{mark}^7$, however we see that even when providing input states, if $f(x)=1$ we will change the value of our input $|y\rangle$.  However, if $f(x)=0$ then neither the input $|x\rangle$ or $|y\rangle$ will be modified by the oracle.  By definition, if the marking oracle evaluates to one for some input, hence if $f(x)=1$ for some $x$, then the input state $|y\rangle$ will be changed.  Specifically, $|y\rangle$ will be flipped.

Now let us define a state $|\alpha\rangle$ such that $|x\rangle$ is a superposition of the $6$ and $7$ basis states and $|y\rangle = |0\rangle$:
$$|\alpha\rangle = \frac{1}{\sqrt{2}}\big(|110\rangle |0\rangle + |111\rangle |0\rangle\big) = $$
$$= |11\rangle \otimes \frac{1}{\sqrt{2}} \big(|0\rangle + |1\rangle\big) \otimes |0\rangle = |11+\rangle |0\rangle$$

How does our marking oracle $U_{7,mark}$ act on our state $|\alpha\rangle$?
> Recall that oracles are simply operators and operators are linear thus they can be distributed to each term individually.

$$U_{7,mark} |\alpha\rangle = \frac{1}{\sqrt{2}} \big(U_{7,mark}|110\rangle |0\rangle + U_{7,mark}|111\rangle |0\rangle\big) =$$
$$= \frac{1}{\sqrt{2}} \big(|110\rangle |0\rangle + |111\rangle |1\rangle\big) := |\epsilon\rangle$$

Now we would like to observe how our input state $|\alpha\rangle$ was modified by the oracle.  Let's simplify the resulting state from the oracle, $|\epsilon\rangle$:
$$|\epsilon\rangle = \frac{1}{\sqrt{2}} \big(|110\rangle |0\rangle + |111\rangle |1\rangle\big) = |11\rangle \otimes \frac{1}{\sqrt{2}} \big(|0\rangle |0\rangle + |1\rangle |1\rangle\big) = $$
$$= |11\rangle \otimes \frac{1}{\sqrt{2}} \big(|00\rangle + |11\rangle\big) = |11\rangle \otimes |\Phi^+\rangle = |11\Phi^+\rangle$$

Here we see that we have now entangled our input states $|x\rangle$ and $|y\rangle$!  This is a common occurrence for marking oracles when the input is a superposition of basis states: after applying the oracle, the input $|x\rangle$ will become entangled with $|y\rangle$.  Here we see that the result is the bell state $|\Phi^+\rangle$.

>As an exercise, what entangled state would we get in the previous example if $|y\rangle = |1\rangle$ instead of $|y\rangle = |0\rangle$?
>
> <br/>
> <details>
>   <summary><b>Click here for the answer!</b></summary>
>   The entangled state that is produced would be $|\Psi^+\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)$.  Overall, we would get the following: $U_{mark}^7 |011+\rangle |1\rangle = |011\rangle |\Psi^+\rangle$
> </details>

# Part II: Phase Kickback

Previously with marking oracles we considered when the register $|x\rangle$ was in a basis state or a superposition state with the target qubit $|y\rangle$ in a basis state $\big( |0\rangle \text{ or } |1\rangle \big)$.  However, how might the effect of our marking oracles change if our target is *also* in a superposition state?  This case is where we will to observe **phase kickback**.

**Phase kickback** is a very important occurrence in quantum computing and is a key ingredient to many quantum algorithms.  When we apply a marking oracle on a register $|x\rangle$ which is a superposition of state with a target qubit $|y\rangle$ which is also in superposition, the phase from $|y\rangle$ will be *kicked back* into our register $|x\rangle$.  

In order to observe phase kickback, we must use the target qubit $|y\rangle=|-\rangle$.  This is the standard choice for two reasons.  First, for phase kickback to occur and be observable, the target qubit must have a difference in phase between the two basis states $|0\rangle$ and $|1\rangle$.  Second, $|y\rangle$ must be an equal superposition, if it's not then the target $|y\rangle$ and the register $|x\rangle$ will become entangled after applying the marking oracle.

Generally, if we have a marking oracle $U_{mark}$ which implements some function $f(x)$ and we apply this marking oracle on the register $|x\rangle$ and target qubit $|y\rangle=|-\rangle$ we will have the following results:
* if $|x\rangle$ is in a basis state:
$$U_{mark} |x\rangle |y\rangle = U_{mark} |x\rangle |-\rangle = (-1)^{f(x)}|x\rangle |-\rangle = (-1)^{f(x)}|x\rangle |y\rangle$$


* if $|x\rangle$ is in a superposition state, say $|x\rangle = \frac{1}{\sqrt{3}} \big(|b_1\rangle + |b_2\rangle + |b_3\rangle \big)$ where $|b_1\rangle, |b_2\rangle$ and $|b_3\rangle$ are the basis states of $U_{mark}$ then:
$$U_{mark} |x\rangle |y\rangle = U_{mark} |x\rangle |-\rangle = U_{mark} \frac{1}{\sqrt{3}} \big(|b_1\rangle + |b_2\rangle + |b_3\rangle \big) |-\rangle = $$
$$ = \frac{1}{\sqrt{3}} \big( U_{mark}|b_1\rangle + U_{mark}|b_2\rangle + U_{mark}|b_3\rangle\big) |-\rangle = $$
$$ = \frac{1}{\sqrt{3}} \big( (-1)^{f(b_1)}|b_1\rangle + (-1)^{f(b_2)}|b_2\rangle + (-1)^{f(b_3)}|b_3\rangle\big) |-\rangle = $$
$$ =\frac{1}{\sqrt{3}} \big( (-1)^{f(b_1)}|b_1\rangle + (-1)^{f(b_2)}|b_2\rangle + (-1)^{f(b_3)}|b_3\rangle\big) |y\rangle$$

From these two cases we see that in the end, $U_{mark}$ does not change the target qubit $|y\rangle$ while it does change the register $|x\rangle$ thus we can drop the target qubit $|y\rangle$ without any repercussions.  Notice that if we do indeed drop the target qubit, we now have the following state:
$$|\psi\rangle = \frac{1}{\sqrt{3}} \big( (-1)^{f(b_1)}|b_1\rangle + (-1)^{f(b_2)}|b_2\rangle + (-1)^{f(b_3)}|b_3\rangle\big)$$

Which looks as if we had just applied a phase oracle to it instead of a marking oracle.  This is one very important application of phase kickback: it allows us to convert between a marking oracle and a phase oracle - which you will implement in the next exercise!  

> Another important application is **phase estimation** which deals with encoding the eigenvalue of an eigenvector into the output state.  Specifically the problem statement is: given $|v\rangle$, an eigenvector of operator $V$ with eigenvalue $e^{\frac{2\pi i \phi}{2^n}}$ for some $\phi$, find the value of $\phi$.  The solution circuit will map the input state $|00...0\rangle \otimes |v\rangle$ to $|\phi\rangle \otimes |v\rangle$ for which you can determine the value of $\phi$ via measurement.  Phase estimation is used to factor integers in polynomial time, a massive result within the field of quantum computation.

To become more familiar with the mathematics of phase estimation we will consider the following example using the oracle you previously implemented: $U_{7,mark}$. Consider beginning with $|x\rangle$ as an equal superposition of the $6$ and $7$ basis states and $|y\rangle=|-\rangle$, our overall state $|\eta\rangle$ is the following:
$$|\eta\rangle = \Big[\frac{1}{\sqrt{2}}\big(|110\rangle + |111\rangle\big)\Big] \otimes \frac{1}{\sqrt{2}}\big(|0\rangle - |1\rangle\big) = $$
$$ = \frac{1}{2} \big(|110\rangle|0\rangle + |111\rangle|0\rangle - |110\rangle|1\rangle - |111\rangle|1\rangle\big)$$

How does $U_{7,mark}$ act on our state $|\eta\rangle$?
$$U_{7,mark}|\eta\rangle = U_{7,mark} \frac{1}{2} \big(|110\rangle|0\rangle + |111\rangle|0\rangle - |110\rangle|1\rangle - |111\rangle|1\rangle \big) = $$
$$= \frac{1}{2} \big( U_{7,mark}|110\rangle|0\rangle + U_{7,mark}|111\rangle|0\rangle - U_{7,mark}|110\rangle|1\rangle - U_{7,mark}|111\rangle|1\rangle \big) = $$
$$= \frac{1}{2} \big(|110\rangle|0\rangle + |111\rangle|1\rangle - |110\rangle|1\rangle - |111\rangle|0\rangle \big) := |\xi\rangle$$

Now we would like to observe how our input state $|\eta\rangle$ was modified by the oracle.  Let's simplify the resulting state from the oracle, $|\xi\rangle$:
$$|\xi\rangle = \frac{1}{2} \big(|110\rangle|0\rangle + |111\rangle|1\rangle - |110\rangle|1\rangle - |111\rangle|0\rangle\big)  = $$
$$= \frac{1}{2} \big(|110\rangle|0\rangle - |110\rangle|1\rangle - |111\rangle|0\rangle + |111\rangle|1\rangle \big) = $$
$$= \frac{1}{2} \Big[|110\rangle \otimes \big(|0\rangle - |1\rangle \big) + |111\rangle \otimes \big(|1\rangle - |0\rangle\big)\Big] = $$
$$ = \Big[\frac{1}{\sqrt{2}} \big( |110\rangle - |111\rangle \big) \Big] \otimes \Big[ \frac{1}{\sqrt{2}} \big( |0\rangle - |1\rangle \big) \Big] = $$
$$= \Big[\frac{1}{\sqrt{2}} \big( |110\rangle - |111\rangle \big) \Big] \otimes |-\rangle$$

Finally lets compare $|\eta\rangle$ and $|\xi\rangle$ directly, below the final equations are repeated for your convenience:
$$|\eta\rangle = \Big[\frac{1}{\sqrt{2}}\big(|110\rangle + |111\rangle\big)\Big] \otimes |-\rangle$$
$$|\xi\rangle = \Big[\frac{1}{\sqrt{2}}\big(|110\rangle - |111\rangle\big)\Big] \otimes |-\rangle$$

We can see that these two equations are identical except for the phase on the $|111\rangle$ basis state (representing $7$).  This is a specific example where phase kickback occurs as the phase from $|y\rangle$ has been *kicked back* into $|x\rangle$.

How could we determine whether or not phase kickback had occured in the previous example?  Consider the following simplification of our final equations first:
$$|\eta\rangle = \Big[\frac{1}{\sqrt{2}}\big(|110\rangle + |111\rangle\big)\Big] \otimes |-\rangle =$$
$$= |11\rangle \otimes \frac{1}{\sqrt{2}}\big( |0\rangle + |1\rangle \big) \otimes |-\rangle = |11+\rangle |-\rangle$$


$$|\xi\rangle = \Big[\frac{1}{\sqrt{2}}\big(|110\rangle - |111\rangle\big)\Big] \otimes |-\rangle =$$
$$= |11\rangle \otimes \frac{1}{\sqrt{2}}\big( |0\rangle - |1\rangle \big) \otimes |-\rangle = |11-\rangle |-\rangle$$

<br/>
<details>
  <summary><b>Now how could we differentiate between the state $|11+\rangle |-\rangle$ and $|11-\rangle |-\rangle$?  Take a moment to think, then click here to see if you were correct!</b></summary>
    That's right!  If we apply a Hadamard operator to the third qubit, we will be able to distinguish between the input state and the output state. $$(I\otimes I \otimes H)|11+\rangle = |110\rangle$$ $$(I\otimes I \otimes H)|11-\rangle = |111\rangle$$ Now if we were to measure the input state versus the output state we would detect that the phase from $|y\rangle=|-\rangle$ was kicked back into our input state $|x\rangle$!
</details>

Now we will explore how you could create a oracle converter

### Task 2.1: Apply the phase oracle
**Inputs:**

  1. A marking oracle implementing some arbitrary function $f(x)$
  2. N qubits in an arbitrary state $|qubits\rangle$ (input/query register)
  
**Goal:**

Flip the sign of the register if $|qubits\rangle$ satisifies the function $f(x)$
that the marking oracle implements.  Specificially, flip the sign of the input
register if $f(|qubits\rangle)=1$.

<br/>
<details>
  <summary><b>If you're stuck on where to begin, click here!</b></summary>
    Recall that you can allocate extra qubits to assist in any operation.  Is there a state that you could prepare with an auxiliary qubit which would enable you to flip the phase of the input state $|x\rangle$ subject to the function $f(x)$ for the marking oracle?
</details>

In [ ]:
%kata T21_ApplyMarkingOracleAsPhaseOracle

operation ApplyMarkingOracleAsPhaseOracle(markingOracle: ((Qubit[], Qubit) => Unit is Adj + Ctl), qubits: Qubit[]) : Unit is Adj + Ctl {
    // ...
}

### Demo 2.1: Oracle Conversion

In this demo we will use your implementation from Task 2.1 to convert the marking oracle that you implemented in Task 1.3 to a phase oracle.  Then we will compare this converted oracle to the phase oracle analog that you implemented in Task 1.2.

> **You must have Tasks 1.2, 1.3, and 2.1 completed correctly for this demo to behave correctly!**

In [ ]:
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Diagnostics;

operation Oracle_Converter_Demo() : Unit {
    let divider = "--------------------------------------------------------------------------------------------------";

    // allocate the state |000⟩
    using (register = Qubit[3]) {
        // convert to the equal superposition state
        ApplyToEachA(H, register);
        
        // dump the state
        Message("the equal superposition register before applying the phase oracle you implemented:");
        DumpMachine();
        Message(divider);
        
        // apply the oracle you implemented:
        IsSeven_PhaseOracle(register);
        
        // dump the state after application of the oracle
        Message("the equal superposition register after applying the phase oracle you implemented:");
        DumpMachine();
        Message(divider);
        
        // reset the qubits for deallocation
        ResetAll(register);
    }
    
    // allocate the state |000⟩
    using (register = Qubit[3]) {
        // convert to the equal superposition state
        ApplyToEachA(H, register);

        // apply the marking oracle as a phase oracle
        ApplyMarkingOracleAsPhaseOracle(IsSeven_MarkingOracle, register);
        
        // dump the state after application of the oracle
        Message("the equal superposition register after applying the converted marking oracle:");
        DumpMachine();
        Message(divider);
        
        // reset the qubits for deallocation
        ResetAll(register);
    }
}

In [ ]:
%simulate Oracle_Converter_Demo

> Notice from the above demo that your phase oracle $U_{7,phase}$ behaves the same as the converted version of your marking oracle $U_{7,mark}$, both of which induce a phase flip on the basis state $|111\rangle$!

> An oracle converter is useful because many quantum algorithms rely on a phase oracle, such as Grover's algorithm, however it is often easier to implement a marking oracle.  This converter will provide us a way to convert a marking oracle we are interested in into a phase oracle; which could then be leveraged in a quantum algorithm such as Grover's algorithm.

# Part III: Implementing Quantum Oracles

In this section you will implement a few quantum oracles of your own - each representing a specified function.  If you observe the operation declarations below, you will see that there is a requirement for the adjoint of the operation to be valid - the operation performed in reverse order must be valid.  This is common practice: when we write quantum oracles, we want to undo any operations we may have performed on the input to achieve the result in order to ensure that the oracle does not change the input if that input is a basis state of the oracle.

### Task 3.1: Implement the OR oracle
Consider an oracle defined as follows:

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)
    
  2. A qubit in an arbitrary state $|y\rangle$ (target qubit)

**Goal:**

Flip the state of $|y\rangle$ if the input register is in any state
except for $|00...0\rangle$ (the all zero state).

**Examples:**

* If the query register is in the state $|10000001\rangle$, flip the state $|y\rangle$.

* If the query register is in the state $|11101101\rangle$, flip the state $|y\rangle$.

* If the query register is in the state $|000\rangle$, do nothing.

* If the query register is in the state $|0010101\rangle$, flip the state $|y\rangle$.

<br/>
<details>
  <summary><b>Before implementing this oracle, answer the following question: are you implementing a marking or a phase oracle?  Click here for the answer!</b></summary>
    Correct!  This is a marking oracle because we are flipping the value of some target qubit $|y\rangle$ based on the value of the other input $|x\rangle$
</details>

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  To solve this problem you need to flip the state of $|y\rangle$ for every input except $|00...0\rangle$.  You may also find the Q# library function ControledOnInt in Microsoft.Quantum.Canon useful in your implementation.
</details>

In [ ]:
%kata T31_Or_Oracle

operation Or_Oracle(x: Qubit[], y: Qubit) : Unit is Adj + Ctl {
    // ...
}

### Task 3.2: Implement the kth bit oracle
Consider an oracle defined as follows:

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)
  
  2. An integer value $k$ such that $0 \leq k < N$

**Goal:**

Flip the sign of the input state $|x\rangle$ if the $k$th qubit in the
input register is a $1$.  

* **Implement this oracle without using auxiliary qubits.**

**Examples:**

* If the query register is in the state $|010\rangle$ and $k=0$, do nothing.

* If the query register is in the state $|101\rangle$ and $k=0$, flip the sign of the register.

* If the query register is in the state $|100101\rangle$ and $k=2$, do nothing.

* If the query register is in the state $|011001\rangle$ and $k=5$, flip the sign of the register.

**TODO: @(William / Mariia) is k measured from the right or the left?**

<br/>
<details>
  <summary><b>Before implementing this oracle, answer the following question: are you implementing a marking or a phase oracle?  Click here for the answer!</b></summary>
    Correct!  This is a phase oracle because we are applying a phase to the input state $|x\rangle$ based on the evaluation of the function $f(x)$.
</details>

In [ ]:
%kata T32_kthBit_Oracle

operation kthBit_Oracle(x: Qubit[], k: Int) : Unit is Adj + Ctl {
    // ...
}

As you noticed in the previous exercise, undoing operations performed on the input is essential to ensure that we are implementing oracles that follow the requirements of an oracle - oracles are unitary and they do not change the input if that input is a basis state.  Another key tool to have when implementing quantum oracles is allocation of auxiliary qubits to assist in a computation.  Below are some exercises where you will practice allocating extra qubits to assist with the computation of $f(x)$.

### Task 3.3: Implement the OR oracle with the kth qubit excluded
Consider an oracle defined as follows:

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)
  
  2. An integer value $k$ such that $0 \leq k < N$

**Goal:**

Flip the sign of the input state $|x\rangle$ if the input register, minus the
$k$th qubit fullfils the requirements of the OR orcale.  Thus if the input
register, with the $k$th qubit excluded, is not the all zero state then flip
the sign of the input register.

**Examples:**

* If the query register is in the state $|010\rangle$ and $k=0$, flip the sign of the register.

* If the query register is in the state $|010\rangle$ and $k=1$, do nothing.

* If the query register is in the state $|100101\rangle$ and $k=2$, flip the sign of the register.

* If the query register is in the state $|011001\rangle$ and $k=5$, flip the sign of the register.

<br/>
<details>
  <summary><b>Before implementing this oracle, answer the following question: are you implementing a marking or a phase oracle?  Click here for the answer!</b></summary>
    Correct!  This is a phase oracle because we are applying a phase to the input state $|x\rangle$ based on the evaluation of the function $f(x)$.
</details>

> Feel free to explore implementing this operation without auxiliary qubits!

In [ ]:
%kata T33_OrOfBitsExceptKth_Oracle

operation OrOfBitsExceptKth_Oracle(x: Qubit[], k: Int) : Unit is Adj + Ctl {
    // ...
}

# Part IV: More Oracles!  Implementation and Testing:

### Task 4.1: Implement the arbitrary bit pattern oracle
Consider an oracle defined as follows:

**Inputs:**

  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)
    
  2. A qubit in an arbitrary state $|y\rangle$ (target qubit)
  
  3. A boolean array of length N representing some arbitrary state

**Goal:**

Flip the state of $|y\rangle$ if the input register matches the state
that the boolean array $pattern$ represents.  Note that a value of true
in the boolean array at some index corresponds to requiring  a $1$ in $|x\rangle$ 
at that index.  For a false in the boolean array at some index, a $0$ is
expected to appear in $|x\rangle$ at that index.

**Examples:**

* If the query register is in the state $|010\rangle$ and $pattern=$[false, true, false], flip the state $|y\rangle$.

* If the query register is in the state $|1001\rangle$ and $pattern=$[false, true, true, false], do nothing.

* If the query register is in the state $|1101\rangle$ and $pattern=$[true, true, false, false], do nothing.

* If the query register is in the state $|01010\rangle$ and $pattern=$[false, true, false, true, false], flip the state $|y\rangle$.
    
<br/>
<details>
  <summary><b>Before implementing this oracle, answer the following question: are you implementing a marking or a phase oracle?  Click here for the answer!</b></summary>
    Correct!  This is a marking oracle because we are flipping the value of some target qubit $|y\rangle$ based on the value of the other input $|x\rangle$
</details>

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  To solve this problem you need to flip the state of $|y\rangle$ if $x\rangle$ matches the pattern laid out in the input parameter $pattern$.  You may also find the Q# library function ControlledOnBitString in Microsoft.Quantum.Canon useful in your implementation.
</details>

In [ ]:
%kata T41_ArbitraryBitPattern_Oracle

operation ArbitraryBitPattern_Oracle(x: Qubit[], y: Qubit, pattern: Bool[]) : Unit is Adj + Ctl {
    // ...
}

> ### Task 4.2: Implement the arbitrary bit pattern oracle
> Consider an oracle defined as follows:
>
> **Inputs:**
>
>  1. N qubits in an arbitrary state $|x\rangle$ (input/query register)
>  
>  2. A boolean array of length N representing some arbitrary state
>
> **Goal:**
> 
> Flip the sign of the input state $|x\rangle$ if the input register matches the state
> that the boolean array $pattern$ represents.  Note that a value of true
> in the boolean array at some index corresponds to requiring  a $1$ in $|x\rangle$ 
> at that index.  For a false in the boolean array at some index, a $0$ is
> expected to appear in $|x\rangle$ at that index.
> 
> * **You must implement this oracle without using auxiliary qubits**
> 
> **Examples:**
>
> * If the query register is in the state $|010\rangle$ and $pattern=$[false, true, false], flip the sign of the input register.
>
> * If the query register is in the state $|1001\rangle$ and $pattern=$[false, true, true, false], do nothing.
>
> * If the query register is in the state $|1101\rangle$ and $pattern=$[true, true, false, false], do nothing.
>
> * If the query register is in the state $|01010\rangle$ and $pattern=$[false, true, false, true, false], flip the sign of the input register.
>
>  
> <br/>
> <details>
>  <summary><b>Before implementing this oracle, answer the following question: are you implementing a marking or a phase oracle?  Click here for the answer!</b></summary>
>    Correct!  This is a phase oracle because we are applying a phase to the input state $|x\rangle$ based on the evaluation of the function $f(x)$.
> </details>

In [ ]:
%kata T42_ArbitraryBitPattern_Oracle_Challenge

operation ArbitraryBitPattern_Oracle_Challenge(x: Qubit[], pattern: Bool[]) : Unit is Adj + Ctl {
    // ...
}

### Task 4.3: Implement the meeting oracle

Suppose that you would like to schedule a meeting with your co-worker Jasmine.  $|x\rangle$ represents your schedule for the five day workweek and $|jasmine\rangle$ represents Jasmine's schedule.  If a $1$ appears in either schedule it means that respective person is busy that day, a $0$ means they're free for a meeting that day.  Implement an oracle that determines if yourself and Jasmine can schedule a meeting.

Consider an oracle defined as follows:

**Inputs:**

  1. 5 qubits in an arbitrary state $|x\rangle$ representing your schedule for the week (input/query register)
  
  2. 5 qubits in an arbitrary state $|jasmine\rangle$ representing Jasmine's schedule for the week (input/query register) 
    
  3. A qubit in an arbitrary state $|z\rangle$ (target qubit)

**Goal:**

Flip the state of $|z\rangle$ if the both you and Jasmine are both free
on the same day of the week.  Recall that a $0$ means that a person is free
on that day.

**Examples:**

* If $|x\rangle=|10101\rangle$ and $|jasmine\rangle=|01010\rangle$, do nothing.

* If $|x\rangle=|10101\rangle$ and $|jasmine\rangle=|01110\rangle$, flip the state $|z\rangle$

* If $|x\rangle=|00000\rangle$ and $|jasmine\rangle=|00000\rangle$, flip the state $|z\rangle$

* If $|x\rangle=|11111\rangle$ and $|jasmine\rangle=|11111\rangle$, do nothing.
    
<br/>
<details>
  <summary><b>Before implementing this oracle, answer the following question: are you implementing a marking or a phase oracle?  Click here for the answer!</b></summary>
    Correct!  This is a marking oracle because we are flipping the value of some target qubit $|z\rangle$ based on the value of the other input $|x\rangle$ and $|y\rangle$.  Notice that here, even though we do not have the typical situation for marking oracles that we saw earlier, this is still a marking oracle.
</details>

In [ ]:
%kata T43_Meeting_Oracle

operation Meeting_Oracle(x: Qubit[], jasmine: Qubit[], z: Qubit) : Unit is Adj + Ctl {
    // ...
}

### Demo 4.1: Testing an Oracle

The purpose of this demo is to demonstrate how you could test an oracle that you've programmed in industry.  For all of the previous oracles that you've implemented, we've been testing your oracle against a reference solution for that oracle.  However, if you're designing an oracle in industry, you do not have a reference solution for the oracle - if you did then there would be no point for you to program the oracle in the first place!

A way to test a quantum oracle of interest is to write a classical oracle that produces the same probability distribution, that way you can compare the output of your quantum oracle with the output of the classical oracle for every input, or a majority of the inputs constrained by runtime, to ensure that they match.

In this demonstration, we will be testing the solution of your oracle $U_{SAT}$ by comparing it to a classical reference solutions' probability distribution. 

In [ ]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Convert;

// the classical oracle to produce the same probability distribution
function Meeting_Classical_Oracle(x: Bool[], jasmine: Bool[]) : Bool {
    for (i in IndexRange(x)) {
        if ((not x[i]) and (not jasmine[i])) {
            // they have a day that they can both meet
            return true;
        }
    }
    
    // they're both busy every day of the week
    return false;
}

operation Test_Meeting_Oracle() : Unit {
    // there are 2^5 ways to arrange a persons schedule
    for (k in 0..((2^5)-1)) { 
        for (j in 0..((2^5)-1)) {
            // prepare your schedule
            let binaryX = IntAsBoolArray(k, 5);
            
            // prepare Jasmine's schedule
            let binaryJasmine = IntAsBoolArray(j, 5);
            
            // create a register of qubits so we can represent
            // your schedule, jasmine's schedule, and the output
            using (register = Qubit[11]) {
                // split the register into the two schedules and the target qubit
                let x = register[0..4];
                let jasmine = register[5..9];
                let target = register[10];
                
                // match the quantum schedules with the binary schedules
                ApplyPauliFromBitString(PauliX, true, binaryX, x);
                ApplyPauliFromBitString(PauliX, true, binaryJasmine, jasmine);
                
                // apply the classical oracle
                let classicalResult = Meeting_Classical_Oracle(binaryX, binaryJasmine);
                
                // apply the quantum oracle
                Meeting_Oracle(x, jasmine, target);
                
                // ensure that the result of the quantum algorithm matched that
                // of the classical algorithm
                AssertQubit(classicalResult ? One | Zero, target);
                
                // undo the prepreation of x and jasmine
                ApplyPauliFromBitString(PauliX, true, binaryX, x);
                ApplyPauliFromBitString(PauliX, true, binaryJasmine, jasmine);
                
                // ensure that the oracle did not change its input states
                AssertAllZero(x);
                AssertAllZero(jasmine);
                
                // ensure that the target is in the zero state so that it can be deallocated
                Reset(target);
            }
        }
    }
    
    Message("Success!");
}

In [ ]:
%simulate Test_Meeting_Oracle

# Part V: What's next?

Thanks for learning with us!  We hope that you enjoyed the tutorial and if you'd like to learn more about quantum computing and Q#, here are some suggestions:

* [The Exploring Deutsch-Jozsa algorithm tutorial](https://github.com/microsoft/QuantumKatas/tree/main/tutorials/ExploringDeutschJozsaAlgorithm) to become more familiar with an application of quantum oracles
* [The Exploring Grover’s search algorithm tutorial](https://github.com/microsoft/QuantumKatas/tree/main/tutorials/ExploringGroversAlgorithm) to interact with another important quantum oracle which is used in many other quantum algorithms to reach a solution.